In [2]:
from pyspark.sql import functions as F

import pandas as pd
import numpy as np

In [2]:
sqlContext.sql("set spark.sql.shuffle.partitions=500")

DataFrame[key: string, value: string]

In [3]:
contest_data = sqlContext.sql("select * from ignite.contest_data_stamped").repartition(200).cache()
so_end_customer = sqlContext.sql('select * from edso_ignite.contest_so_end_customer').repartition(200).cache()
sol_end_customer = sqlContext.sql('select * from edso_ignite.contest_sol_end_customer').repartition(200).cache()
party_info = sqlContext.sql("select * from edso_ignite.contest_party_info").repartition(200).cache()
end_customer = sqlContext.sql('select * from edso_ignite.contest_end_customer').repartition(200).cache()
cr_parsed = sqlContext.sql('select * from ignite.cr_parsed').repartition(200).cache()
savm_parsed = sqlContext.sql("select * from ignite.savm_parsed").repartition(500, 'creation_date').cache()

In [4]:
def drop_columns(df, columns):
    return df.select([c for c in df.columns if c not in columns])

In [5]:
party_values = [
    'end_customer_party_ssot_party_id_int_sav_party_id',
    'prior_party_ssot_party_id_int_sav_party_id',
    'sol_branch_party',
    'sol_gu_party',
    'sol_hq_party',
    'order_level_branch_party',
    'order_level_gu_party',
    'order_level_hq_party',
    'line_level_branch_party',
    'line_level_gu_party',
    'line_level_hq_party',
    'ship_to_branch_party',
    'ship_to_gu_party',
    'ship_to_hq_party',
    'bill_to_branch_party',
    'bill_to_gu_party',
    'bill_to_hq_party',
    'sold_to_branch_party',
    'sold_to_gu_party',
    'sold_to_hq_party'
]

In [10]:
party_expansion = sqlContext.sql("select * from ignite.party_expansion")

In [7]:
flattened_parties = party_expansion.map(lambda x : (x.id, 
                                                  list(set([
            x.end_customer_party_ssot_party_id_int_sav_party_id,
            x.prior_party_ssot_party_id_int_sav_party_id,
            x.pppk_party,
            x.sol_branch_party,
            x.sol_gu_party,
            x.sol_hq_party,
            x.order_level_branch_party,
            x.order_level_gu_party,
            x.order_level_hq_party,
            x.line_level_branch_party,
            x.line_level_gu_party,
            x.line_level_hq_party,
            x.ship_to_branch_party,
            x.ship_to_gu_party,
            x.ship_to_hq_party,
            x.bill_to_branch_party,
            x.bill_to_gu_party,
            x.bill_to_hq_party,
            x.sold_to_branch_party,
            x.sold_to_gu_party,
            x.sold_to_hq_party
        ])))).flatMapValues(lambda x : x).toDF(['id', 'candidate_party']).drop_duplicates()

In [8]:
all_sales_accts = flattened_parties.repartition(500, 'id').join(party_expansion.repartition(500, 'id').select(['id', 'decision_date_time']), on = 'id').cache()

In [20]:
joining_savm_party = savm_parsed.select(['party_id', 'creation_date', F.col('sales_acct_id').alias('candidate_sales_acct_id')]).drop_duplicates()
builder_party = all_sales_accts.join(joining_savm_party, on = (F.col('party_id') == F.col('candidate_party')))
#builder_party = builder_party.where(F.col('creation_date') < F.col('decision_date_time')).drop_duplicates()
builder_party = builder_party.select(['id', 'candidate_party', 'candidate_sales_acct_id'])
builder_party.cache()

DataFrame[id: bigint, candidate_party: bigint, candidate_sales_acct_id: double]

In [13]:
joining_savm_party = savm_parsed.select(['parent_party_id', F.col('sales_acct_id').alias('candidate_sales_acct_id')]).drop_duplicates()
parent_party_candidates = all_sales_accts.join(joining_savm_party, on = (F.col('parent_party_id') == F.col('candidate_party')))
parent_party_candidates = parent_party_candidates.drop_duplicates()
parent_party_candidates.cache()

DataFrame[id: bigint, candidate_party: bigint, decision_date_time: date, parent_party_id: double, candidate_sales_acct_id: double]

In [16]:
joining_savm_party.count()

12190957

In [17]:
builder_party.count()

10525554

In [14]:
parent_party_candidates.count()

34548921

In [8]:
builder_party.write.saveAsTable('ignite.direct_id_candidates')

NameError: name 'builder_party' is not defined

In [ ]:
parent_party_candidates.write.saveAsTable('ignite.parent_expansion_candidates')

In [11]:
id_candidates = builder_party.unionAll(parent_party_candidates)

In [23]:
id_candidates.count()

45220456

In [21]:
joining_contest_data = contest_data.select([F.col('sales_acct_id').alias('truth_sales_acct_id'), 'id'])
correct_candidates = joining_contest_data.join(builder_party, on = 'id').where(F.col('truth_sales_acct_id') == F.col('candidate_sales_acct_id'))
distinct_candidates = correct_candidates.select(['id']).distinct()
unmatched_ids = distinct_candidates.select([F.col('id').alias('generated_id'), 'id']).join(contest_data.select(['id']), on = 'id', how = 'right').where(F.col('generated_id').isNull())

In [22]:
correct_candidates.select(['id']).distinct().count()

674906

In [30]:
def correct_candidates(candidate_set):
    joining_contest_data = contest_data.select([F.col('sales_acct_id').alias('truth_sales_acct_id'), 'id'])
    correct_candidates = joining_contest_data.join(candidate_set, on = 'id').where(F.col('truth_sales_acct_id') == F.col('candidate_sales_acct_id'))
    return correct_candidates.select(['id']).distinct()

In [7]:
id_candidates = sqlContext.sql('select * from ignite.direct_id_candidates').unionAll(sqlContext.sql('select * from ignite.parent_expansion_candidates')).repartition(200).cache()

In [7]:
contest_data.where(F.col('id') == 14431).take(1)

[Row(endcustomerlinefixed=u'LOH SERVICES GMBH & CO. KG', pppk=5270081, fppk=106855484, decision_date_time=u'2016-03-16 06:44:50.0', days_from_create_to_final_decision=20.291666, end_customer_party_ssot_party_id_int_sav_party_id=41313822, prior_party_ssot_party_id_int_sav_party_id=41313822, final_party_ssot_party_id_int_sav_party_id=174568531, type=u'Reviewed_and_modified', priorpartyname=u'LOH SERVICES GMBH & CO. KG', finalpartyname=u'LOH SERVICES', sales_order_key=25309434, sales_order_line_key=254830200, so_number_int=101993209, end_customer_key_line_level=69154289, end_customer_key_order_level=69154289, dd_end_customer_type_order_level=u'ACCOUNT_LOCATION                                  ', purchase_order_type_code=u'Resale', purchase_order_number=u'6550170144', order_datetime=u'2016-02-24 23:46:56.0', oracle_book_datetime=u'2016-02-25 01:25:47.0', transactional_currency_code=u'USD            ', customer_service_rep_name=u'null', cust_svc_rep_team_name=u'null', conversion_type_code=u

In [11]:
party_expansion.where(F.col('sales_acct_id') == 213297969.0).take(5)

[Row(sold_to_customer_key=58758751, bill_to_customer_key=58869428, ship_to_customer_key=69182435, end_customer_key_line_level=69154289, end_customer_key_order_level=69154289, sales_order_line_key=254976245, sales_order_key=25320374, endcustomerlinefixed=u'LOH SERVICES GMBH & CO. KG', pppk=5270081, fppk=106855484, decision_date_time=datetime.date(2016, 3, 16), days_from_create_to_final_decision=19.041666, end_customer_party_ssot_party_id_int_sav_party_id=41313822, prior_party_ssot_party_id_int_sav_party_id=41313822, final_party_ssot_party_id_int_sav_party_id=174568531, type=u'Reviewed_and_modified', priorpartyname=u'LOH SERVICES GMBH & CO. KG', finalpartyname=u'LOH SERVICES', so_number_int=102003525, dd_end_customer_type_order_level=u'ACCOUNT_LOCATION                                  ', purchase_order_type_code=u'Service Provision Use', purchase_order_number=u'6550170722', order_datetime=u'2016-02-26 05:29:43.0', oracle_book_datetime=u'2016-02-26 12:23:18.0', transactional_currency_code

In [12]:
savm_parsed.where(F.col('sales_acct_id') == 215248249.0).take(5)

[Row(party_id=46056212.0, parent_party_id=45471.0, party_name=u'AT & T GLOBAL NETWORK SERVICES', node_type=u'BR', address1=u'C O SENSIENT TECHNOLOGIES', address2=u'100 VULCAN ST', address3=None, address4=None, city=u'TORONTO', county=None, state=u'ON', province=None, postal_code=u'M9W 1L2', postal_code_extn=None, country_code=u'CA', street_name=u'VULCAN', street_number=u'100', street_direction=None, street_type=u'ST', geo_valid_status=u'GEO_VALID', completenes_status=u'COMPLETE', cleansed_status=u'CLEANSED', start_date=u'2007-04-28 08:08:38.0', end_date=u'4712-12-31 00:00:01.0', program_id=None, request_id=None, created_by=307958.0, last_updated_by=307958.0, creation_date=datetime.date(2007, 4, 28), last_update_date=datetime.date(2007, 4, 28), certified_date=u'2014-09-23 12:31:55.0', site_expl_id=423377878.0, conflict_batch_id=75350641.0, sa_member_id=3348946.0, parent_sa_member_id=-1.0, party_level=2.0, link_party_id=45471.0, link_party_type=u'ORGANIZATION', split_pct=100.0, sales_acc